<a href="https://colab.research.google.com/github/hannahkates/nyu-python-public-policy/blob/master/hw_3_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NYU Wagner - Python Coding for Public Policy**
# Solutions to HW 3: Using keywords to categorize 311 requests

## Start by importing necessary packages

In [0]:
import pandas as pd
from google.colab import drive

In [0]:
# You can use pd.set_option() to make sure you see all the rows and columns in your dataframe
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000

## Read and save our cleaned 311 Service Requests dataset as a pandas dataframe named "df"

In [0]:
drive.mount('/content/drive')
# follow the link it generates, choose your account, and then paste in the authorization code it provides

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df = pd.read_csv('/content/drive/My Drive/Data for Python/cleaned_311_data_hw3.csv', header='infer')

## **Problem Statement:** 
When you read through the `descriptor` column in the 311 data, you will see that complaints related to graffiti are actually scattered throughout multiple `complaint_type` categories. We want to identify all complaints related to graffiti and see which community districts have the most instances of graffiti.

**Step 1**. Create a function that checks each row in the 311 dataframe to see if the word "graffiti" is present in either the `complaint_type` value or `descriptor` value. Both columns may contain the word, so you should check both. If the word "graffiti" is found, the function should return the boolean value True. If "graffiti" is not found, the function should return the boolean value False.

In [0]:
def flag_graffiti(row):
  if 'graffiti' in row.complaint_type.lower():
    return True
  elif pd.notnull(row.descriptor):
    if 'graffiti' in row.descriptor.lower():
      return True
    else:
      return False
  else:
    return False

**Step 2**. Apply the function to the 311 dataframe and create a new column called `graffiti_flag` that captures the output from the function.

In [0]:
df['graffiti_flag'] = df.apply(flag_graffiti, axis=1)

In [0]:
df.head()

,Unnamed: 0,Unnamed: 0.1,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,incident_address,street_name,cross_street_1,cross_street_2,intersection_street_1,intersection_street_2,address_type,city,landmark,facility_type,status,due_date,resolution_description,resolution_action_updated_date,community_board,bbl,borough,x_coordinate_(state_plane),y_coordinate_(state_plane),open_data_channel_type,park_facility_name,park_borough,vehicle_type,taxi_company_borough,taxi_pick_up_location,bridge_highway_name,bridge_highway_direction,road_ramp,bridge_highway_segment,latitude,longitude,location,grafitti_flag,graffiti_flag
0,238,238,39887282,08/01/2018 01:42:01 AM,08/14/2018 02:42:12 PM,DPR,Department of Parks and Recreation,Maintenance or Facility,Structure - Outdoors,Park,10003.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NEW YORK,NaN,NaN,Closed,08/15/2018 01:42:01 AM,The Department of Parks and Recreation has com...,08/14/2018 02:42:12 PM,0 Unspecified,NaN,Unspecified,NaN,NaN,PHONE,First Park,Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
1,353,353,39889407,08/01/2018 03:20:07 AM,08/23/2018 11:15:00 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,11434.0,NaN,NaN,NaN,NaN,BAISLEY BOULEVARD,MERRICK BOULEVARD,INTERSECTION,JAMAICA,NaN,NaN,Closed,08/04/2018 03:20:07 AM,The Service Request submitted did not have suf...,08/23/2018 11:15:00 AM,12 QUEENS,NaN,QUEENS,1048111.0,188479.0,PHONE,Unspecified,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.683778,-73.769744,"(40.68377771085733, -73.76974413109498)",False,False
2,403,403,39893067,08/01/2018 04:19:15 AM,08/02/2018 08:19:55 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Hours of Operation,Park,11206.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BROOKLYN,NaN,NaN,Closed,08/04/2018 04:19:15 AM,The Department of Parks and Recreation has com...,08/02/2018 08:19:55 AM,01 BROOKLYN,NaN,BROOKLYN,NaN,NaN,MOBILE,Ten Eyck Plaza,BROOKLYN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
3,655,655,39890578,08/01/2018 07:32:33 AM,08/16/2018 10:28:38 AM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,11691.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FAR ROCKAWAY,NaN,NaN,Closed,08/04/2018 07:32:33 AM,The Department of Parks and Recreation has com...,08/16/2018 10:28:38 AM,14 QUEENS,NaN,QUEENS,NaN,NaN,PHONE,Westbourne Playground,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
4,658,658,39896325,08/01/2018 07:33:12 AM,09/04/2018 05:38:02 PM,DPR,Department of Parks and Recreation,Maintenance or Facility,Garbage or Litter,Park,11355.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FLUSHING,NaN,NaN,Closed,08/04/2018 07:33:12 AM,The condition was inspected and it was determi...,09/04/2018 05:38:02 PM,07 QUEENS,NaN,QUEENS,NaN,NaN,MOBILE,Maple Playground,QUEENS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [0]:
df.groupby('graffiti_flag').size()

graffiti_flag
False    38206
True     26371
dtype: int64

In [0]:
df.groupby('complaint_type').size()

complaint_type
Bridge Condition                392
Broken Parking Meter          14210
Bus Stop Shelter Complaint      990
Graffiti                      24986
Highway Condition              3543
Maintenance or Facility       17665
Public Payphone Complaint       378
Public Toilet                    41
School Maintenance             2372
dtype: int64

**Step 3**. Create another dataframe `df_graffiti` that only contains records where `graffiti_flag` is True

In [0]:
df_graffiti = df[df['graffiti_flag']]

In [0]:
len(df_graffiti)

26371

**Step 4**. Group your dataframe `df_graffiti` to get the count of requests per `community_board`. Use `.nlargest()` to identify which Community District has the highest count.

In [0]:
df_graffiti.groupby('community_board').size().nlargest(15)

community_board
01 BROOKLYN             1797
03 MANHATTAN            1593
04 BROOKLYN             1429
03 BROOKLYN             1252
05 BROOKLYN              962
07 BROOKLYN              779
Unspecified BRONX        615
02 MANHATTAN             598
Unspecified BROOKLYN     591
01 QUEENS                572
0 Unspecified            540
12 QUEENS                539
12 BROOKLYN              532
14 BROOKLYN              532
08 BROOKLYN              505
dtype: int64

Brooklyn 1 has the highest count of graffiti complaints, followed by Manhattan 3.